In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
import h5py
import pandas as pd
sns.set_color_codes()

# Data generation

In [ ]:
pk_list = []
for loc in (2, 4, 8):
    pk_list.append(np.random.normal(loc, 0.3, 500 * loc))
    
pk_list.append(np.random.uniform(0, 12, 5000))

xs = np.concatenate(pk_list)
del pk_list

In [ ]:
with h5py.File('example_data.h5', 'w') as f:
    f['data'] = xs

# Read data

In [ ]:
with h5py.File('example_data.h5', 'r') as f:
    raw_data = f['data'][:5]

Create a histogram. Put into a DataFrame.

In [ ]:
counts, bins, fig = plt.hist(raw_data, bins=100, histtype='stepfilled')
spectrum = pd.DataFrame(counts, columns=('counts',), index=bins[:-1])

In [ ]:
spectrum.head()

In [ ]:
plt.step(spectrum.index, spectrum.counts)

# Baseline subtraction

In [ ]:
baseline = spectrum.counts.median()
spectrum['bs_sub'] = spectrum.counts - baseline

In [ ]:
spectrum.head()

In [ ]:
plt.step(spectrum.index, spectrum.bs_sub)

# Apply a threshold

In [ ]:
spectrum['clean'] = np.where(spectrum.bs_sub > 15, spectrum.bs_sub, 0)

In [ ]:
spectrum.head()

In [ ]:
plt.step(spectrum.index, spectrum.clean)

# Find peaks

In [ ]:
from scipy.signal import argrelmax

In [ ]:
maxlocs = argrelmax(spectrum.clean.values, order=10)[0]
maxima = spectrum.iloc[maxlocs]

In [ ]:
maxima

In [ ]:
plt.step(spectrum.index, spectrum.clean)
plt.plot(maxima.index, maxima.clean + 0.05 * spectrum.clean.max(), 'rv', markersize=10)

In [ ]:
spectrum['peak_num'] = np.nan
for pknum, (en, ct) in enumerate(maxima.iterrows()):
    spectrum.loc[np.abs(spectrum.index - en) < 1, 'peak_num'] = pknum

In [ ]:
spectrum.dropna().head()

In [ ]:
plt.step(spectrum.index, spectrum.clean, 'k')
palette = sns.color_palette()
for pknum in spectrum.peak_num.dropna().unique():
    plt.fill_between(spectrum.index, spectrum.clean, 
                     where=(spectrum.peak_num == pknum), step='pre', 
                     facecolor=palette[int(pknum)])

# Fit peaks with a Gaussian

Normal distribution:
$$f(x) = \frac{A}{\sigma\sqrt{2\pi}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$$

In [ ]:
def gaus(x, a, m, s):
    return a / (s * np.sqrt(2*np.pi)) * np.exp(- (x - m)**2 / (2 * s**2))

In [ ]:
pknums = spectrum.peak_num.dropna().unique().astype('int')
peaks = pd.DataFrame(columns=('amplitude', 'center', 'sigma'), index=pknums)

for pknum in pknums:
    fitdata = spectrum[spectrum.peak_num == pknum]
    fparams, fcov = curve_fit(gaus, fitdata.index.values, fitdata.clean.values, 
                              p0=(maxima.counts.values[pknum], maxima.index.values[pknum], 0.5))
    peaks.loc[pknum] = fparams

In [ ]:
peaks

In [ ]:
xs = np.linspace(0, 12, 1000)

for pknum in pknums:
    fparams = peaks.iloc[pknum]
    plt.plot(xs, gaus(xs, *fparams))
    plt.fill_between(spectrum.index, spectrum.clean, 
                     where=(spectrum.peak_num == pknum), step='pre', 
                     facecolor=palette[pknum], alpha=0.3)